In [2]:
'''
Advanced Programming 
COM00142M
Summative Assignment
January 2021
'''
# Import modules/libraries
import tkinter as tk
from tkinter import ttk
import tkinter.messagebox
import tkinter.filedialog
import matplotlib.pyplot as plt
%matplotlib tk
import os
import csv
import json
import pandas as pd
import numpy as np
import pymongo


# Main application class
class App:
    def __init__(self, master):
        self.master = master
        
        # TKINTER FILE PATH VARIABLES
        self.filepath_inventory = None
        self.filepath_inspections = None
        self.filepath_violations = None
        
        # PANDAS DATAFRAME VARIABLE 
        self.df_inventory = None
        self.df_inspections = None
        self.df_violations = None
        
        # DATABASE VARIABLES
        self.client = None
        self.collection_inventory = None
        self.collection_inspections = None
        self.collection_violations = None

        # CONTAINER 1 - LEFT SIDE
        self.container_left = tk.Frame(self.master)
        self.container_left.grid(row=0, column=0, sticky='NSEW', padx=5, pady=5, ipadx=5, ipady=5)
        
         # CONTAINER 2 - RIGHT SIDE
        self.container_right = tk.Frame(self.master)
        self.container_right.grid(row=0, column=1, sticky='NSEW', padx=5, pady=5, ipadx=5, ipady=5)
        
        # SECTION 1. LOAD FILES
        self.section_1 = tk.LabelFrame(self.container_left, text=" 1. Load CSV files ")
        self.section_1.grid(row=0, columnspan=10, sticky='W', padx=5, pady=5, ipadx=5, ipady=5)
        
        self.label_1_1 = tk.Label(self.section_1, text="Inventory")
        self.label_1_2 = tk.Label(self.section_1, text="Inspections")
        self.label_1_3 = tk.Label(self.section_1, text="Violations")
        self.label_1_1.grid(row=0, column=0, sticky="W")
        self.label_1_2.grid(row=1, column=0, sticky="W")
        self.label_1_3.grid(row=2, column=0, sticky="W")
        
        self.entry_1_1 = tk.Entry(self.section_1)
        self.entry_1_2 = tk.Entry(self.section_1)
        self.entry_1_3 = tk.Entry(self.section_1)
        self.entry_1_1.insert(0, '...')
        self.entry_1_2.insert(0, '...')
        self.entry_1_3.insert(0, '...')
        self.entry_1_1.grid(row=0, column=1, sticky="W", padx=(5, 5))
        self.entry_1_2.grid(row=1, column=1, sticky="W", padx=(5, 5))
        self.entry_1_3.grid(row=2, column=1, sticky="W", padx=(5, 5))

        self.button_1_1 = tk.Button(self.section_1,text='Browse ...', command= lambda: self.get_path(0))
        self.button_1_2 = tk.Button(self.section_1, text='Browse ...', command= lambda: self.get_path(1))
        self.button_1_3 = tk.Button(self.section_1, text='Browse ...', command= lambda: self.get_path(2))
        self.button_1_1.grid(row=0, column=2, sticky="W", padx=(5, 5))
        self.button_1_2.grid(row=1, column=2, sticky="W", padx=(5, 5))
        self.button_1_3.grid(row=2, column=2, sticky="W", padx=(5, 5))
        self.button_1_load = tk.Button(self.section_1, text='Load', command=self.check_filepaths)
        self.button_1_load.grid(row=3, column=2, sticky="E", padx=(5, 5))
        
        # SECTION 2. CLEAN AND MANIPULATE DATA
        self.section_2 = tk.LabelFrame(self.container_left, text=" 2. Clean and Manipulate Data ")
        self.section_2.grid(row=1, columnspan=10, sticky='W', padx=5, pady=5, ipadx=5, ipady=5)
        
        self.label_2_1 = tk.Label(self.section_2, text="Remove INACTIVE Programs from data")
        self.label_2_2 = tk.Label(self.section_2, text="Extract SEATING information from data")
        self.label_2_1.grid(row=0, column=0, sticky="W")
        self.label_2_2.grid(row=1, column=0, sticky="W")
        
        self.button_2_1 = tk.Button(self.section_2, text='Go', command=self.remove_inactive)
        self.button_2_2 = tk.Button(self.section_2, text='Go', command=self.extract_seating)
        self.button_2_1.grid(row=0, column=4, sticky="E", padx=(5, 5))
        self.button_2_2.grid(row=1, column=4, sticky="E", padx=(5, 5))
        
        # SECTION 3. CONVERT DATA
        self.section_3 = tk.LabelFrame(self.container_left, text=" 3. Convert to JSON Data ")
        self.section_3.grid(row=2, columnspan=10, sticky='W', padx=5, pady=5, ipadx=5, ipady=5)
        
        self.checkbox_var_3_1 = tk.IntVar()
        self.checkbox_var_3_2 = tk.IntVar()
        self.checkbox_var_3_3 = tk.IntVar()
        
        self.checkbox_3_1 = tk.Checkbutton(self.section_3, text="Inventory.csv", variable=self.checkbox_var_3_1)
        self.checkbox_3_2 = tk.Checkbutton(self.section_3, text="Inspections.csv", variable=self.checkbox_var_3_2)
        self.checkbox_3_3 = tk.Checkbutton(self.section_3, text="Violations.csv", variable=self.checkbox_var_3_3)
        self.checkbox_3_1.grid(row=0, column=0, sticky="W")
        self.checkbox_3_2.grid(row=0, column=1, sticky="W")
        self.checkbox_3_3.grid(row=0, column=2, sticky="W")
        
        self.button_3_convert = tk.Button(self.section_3, text='Convert', command=self.check_checkboxes)
        self.button_3_convert.grid(row=0, column=4, sticky="E", padx=(5, 5))
        
        # SECTION 4. DATABASE
        self.section_4 = tk.LabelFrame(self.container_left, text=" 4. MongoDB Database ")
        self.section_4.grid(row=3, columnspan=10, sticky='W', padx=5, pady=5, ipadx=5, ipady=5)
        
        self.Button_4_1_connect = tk.Button(self.section_4, text='Connect', command=self.connect_to_db)
        self.Button_4_1_connect.grid(row=0, column=0, sticky="E", padx=(5, 5))
        
        self.button_4_2_send = tk.Button(self.section_4, text='Store', command=self.send_to_db)
        self.button_4_2_send.grid(row=0, column=1, sticky="E", padx=(5, 5))
        
        self.button_4_2_retrieve = tk.Button(self.section_4, text='Retrieve', command=self.retrieve_from_db)
        self.button_4_2_retrieve.grid(row=0, column=2, sticky="E", padx=(5, 5))
        
        # SECTION 5. PREVIEW DATAFRAME
        self.section_5 = tk.LabelFrame(self.container_left, text=" 5. Preview Dataframes ")
        self.section_5.grid(row=4, columnspan=10, sticky='W', padx=5, pady=5, ipadx=5, ipady=5)
        
        self.button_5_1 = tk.Button(self.section_5, text='Inventory', command= lambda: self.preview_dataframes(0))
        self.button_5_1.grid(row=0, column=0, sticky="E", padx=(5, 5))
        
        self.button_5_2 = tk.Button(self.section_5, text='Inspections', command= lambda: self.preview_dataframes(1))
        self.button_5_2.grid(row=0, column=1, sticky="E", padx=(5, 5))
        
        self.button_5_3 = tk.Button(self.section_5, text='Violations', command= lambda: self.preview_dataframes(2))
        self.button_5_3.grid(row=0, column=2, sticky="E", padx=(5, 5))
        
        # SECTION 6. STATS 
        self.section_6 = tk.LabelFrame(self.container_left, text=" 6. Generate Statistics ")
        self.section_6.grid(row=5, columnspan=10, sticky='W', padx=5, pady=5, ipadx=5, ipady=5)
        
        self.button_6_1 = tk.Button(self.section_6, text='Seating Type', command=self.seating_stats)
        self.button_6_1.grid(row=0, column=0, sticky="E", padx=(5, 5))
        self.button_6_2 = tk.Button(self.section_6, text='Zipcode', command=self.zipcode_stats)
        self.button_6_2.grid(row=0, column=1, sticky="E", padx=(5, 5))

        # SECTION 7. VISUALIZE DATA 
        self.section_7 = tk.LabelFrame(self.container_left, text=" 7. Visualize Data ")
        self.section_7.grid(row=6, columnspan=10, sticky='W', padx=5, pady=5, ipadx=5, ipady=5)
        
        self.button_7_1 = tk.Button(self.section_7, text='Vendors/Violation', command=self.plot_vendors_violations)
        self.button_7_1.grid(row=0, column=0, sticky="E", padx=(5, 5))
        self.button_7_2 = tk.Button(self.section_7, text='Vendors/Zipcode', command=self.plot_vendors_zipcodes)
        self.button_7_2.grid(row=0, column=1, sticky="E", padx=(5, 5))

        # RIGHT SIDE - TABLE VIEW
        self.my_tree = ttk.Treeview(self.container_right, height=30)
        self.scrollbar = tk.Scrollbar(self.container_right, orient='vertical')
        
        #STATUS BAR
        self.my_string_var = tk.StringVar()
        self.my_string_var.set('Browse file paths...')
        
        self.statusbar = tk.Label(self.master, textvariable=self.my_string_var, relief=tk.SUNKEN, anchor=tk.W, bd=2, bg="white")
        self.statusbar.grid(row=1, column=0, columnspan=10, sticky="EW")
    
    # Status Bar - set text and background colour
    def status_bar(self, message, colour):
        self.message = message
        self.colour = colour
        self.my_string_var.set(self.message) # Update label text
        self.statusbar.config(bg=self.colour) # Update label background colour
        self.master.update() # Update GUI
    
    # Get path - open file dialog box and return relative path
    def get_path(self, n):
        self.n = n
        
        # Open dialog box and only allow .csv extensions to be selected
        self.filepath = tkinter.filedialog.askopenfilename(parent=root,
                                                           title='Choose file',
                                                           filetypes=[('csv data', '.csv')])
        if self.n == 0:
            print("Choose a file path for Inventory.csv")
            self.status_bar("Choose a file path for Inventory.csv", "white")
            self.entry_1_1.delete(0, 'end')
            self.entry_1_1.insert(0, self.filepath) # display path in entry box
            self.filepath_inventory = self.filepath # set dataframe as global

        elif self.n == 1:
            print("Choose a file path for Inspections.csv")
            self.status_bar("Choose a file path for Inspections.csv", "white")
            self.entry_1_2.insert(0, self.filepath) # display path in entry box
            self.filepath_inspections = self.filepath # set dataframe as global
            
        elif self.n == 2:
            print("Choose a file path for Violations.csv")
            self.status_bar("Choose a file path for Violations.csv", "white")
            self.entry_1_3.insert(0, self.filepath) # display path in entry box
            self.filepath_violations = self.filepath # set dataframe as global
            
    # Check Filepaths are OK    
    def check_filepaths(self):
        self.empty_strings = ["...", ""] # list of default empty strings
        
        # Get filepath strings from entry box
        self.entry_list = [self.entry_1_1.get(),
                           self.entry_1_2.get(),
                           self.entry_1_3.get()] 
        
        # Check if any of the filepath strings are emtpy
        if any(x in self.entry_list for x in self.empty_strings):
            self.status_bar("Please select the csv filepaths...", "red")  
        else:
            print("File paths OK")
            self.status_bar("File paths OK", "green")
            self.load_csv_files()
    
    # Load CSV data into Pandas DataFrames      
    def load_csv_files(self):
        
        # Load Inventory data into Pandas dataframe and store in global
        print("Loading Inventory.csv ...")
        path, filename = os.path.split(self.filepath_inventory)
        self.df_inventory = pd.read_csv(self.filepath_inventory, encoding="utf-8-sig")
        self.status_bar("Loading Inventory.csv...", "white")
        
        # Load Inspections data into Pandas dataframe and store in global
        print("Loading Inspections.csv ...")
        path, filename = os.path.split(self.filepath_inspections)
        self.df_inspections = pd.read_csv(self.filepath_inspections, encoding="utf-8-sig")
        self.status_bar("Loading Inspections.csv...", "white")
        
        # Load Violations data into Pandas dataframe and store in global
        print("Loading Violations.csv ...")
        path, filename = os.path.split(self.filepath_violations)
        self.df_violations = pd.read_csv(self.filepath_violations, encoding="utf-8-sig")
        self.status_bar("Loading Violations.csv...", "white")
        
        print("Finished loading all datasets")
        self.status_bar("Finished loading all datasets", "green")
    
    # Pre-processing and cleaning the data
    def remove_inactive(self):
        
        # Check is dataframe exits
        if self.df_inspections is None:
            self.status_bar("Please loads the csv data first...", "red")
            print("Need to load csv data first")
        else:
            df = self.df_inspections
            
            # Drop empty rows from critical labels
            print("Removing empty rows...")
            self.status_bar("Removing empty rows...", "white")
            self.df_inspections = self.df_inspections.dropna(subset=['ACTIVITY DATE',
                                                                     'SERIAL NUMBER',
                                                                     'FACILITY ID',
                                                                     'FACILITY ZIP',
                                                                     'PROGRAM STATUS',
                                                                     'PE DESCRIPTION',
                                                                     'SCORE'])
            
            # Cast scores & zipcodes from strings to integers
            print("Casting strings to ints...")
            self.status_bar("Casting strings to ints...", "white")
            self.df_inspections['SCORE'] = self.df_inspections['SCORE'].astype(int)
            self.df_inspections['FACILITY ZIP'] = self.df_inspections['FACILITY ZIP'].astype(str)
            
            # Cast activity date string to a Pandas date object and create a YEAR column
            print("Converting strings date objects...")
            self.status_bar("Converting strings date objects...", "white")
            self.df_inspections['ACTIVITY DATE'] = pd.to_datetime(self.df_inspections['ACTIVITY DATE'])
            self.df_inspections['YEAR'] = self.df_inspections['ACTIVITY DATE'].dt.year
            
            # Regex to shorten Zipcodes to 5 digits
            print("Shortening ZIP code...")
            self.status_bar("Shortening ZIP code...", "white")
            self.df_inspections['temp'] = self.df_inspections['FACILITY ZIP'].str.extract(r"(\d{5})")
            del self.df_inspections['FACILITY ZIP']
            self.df_inspections = self.df_inspections.rename({'temp': 'FACILITY ZIP'}, axis=1)
            
            # Filter out INACTIVE programs
            print("Filtering out INACTIVE programs...")
            self.status_bar("Filtering out INACTIVE programs...", "white")
            #filter out inactive programs status
            self.df_inspections = self.df_inspections.loc[self.df_inspections['PROGRAM STATUS'] == "ACTIVE"]
            self.df_inspections.reset_index(drop=True, inplace=True)
            
            print("INACTIVE programs successfully filtered")
            self.status_bar("INACTIVE programs successfully filtered", "green")

    # Extract seating information and store in new column        
    def extract_seating(self):
        
        # Check that dataframe exists
        if self.df_inspections is None:
            self.status_bar("Please loads the csv data first...", "red")
            print("Please loads the csv data first...")
        else:
            print("Extracting seating information...")
            self.status_bar("Extracting seating information...", "white")
            
            # Regex to extract Seating numbers from label and store in new column SEATING
            self.df_inspections['SEATING'] = self.df_inspections['PE DESCRIPTION'].str.extract('\((.*?)\)', expand=False)
            self.df_inspections['temp'] = self.df_inspections['PE DESCRIPTION'].str.replace('\((.*?)\)',"")
            del self.df_inspections['PE DESCRIPTION']
            self.df_inspections = self.df_inspections.rename({'temp': 'PE DESCRIPTION'}, axis=1) 

            print("Finished extracting seating")
            self.status_bar("SEATING extracted successfully", "green")
    
    # Check boxes to select which dataset to convert to JSON files
    def check_checkboxes(self):
        print("Checking boxes...")
        self.status_bar("Checking boxes...", "white")
        self.checkbox_list = [self.checkbox_var_3_1.get(), self.checkbox_var_3_2.get(), self.checkbox_var_3_3.get()]
        self.df_list = [self.df_inventory, self.df_inspections, self.df_violations]
        
        # Check that all boxes have been checked
        if any(dfs is None for dfs in self.df_list):
            print("Please load csv files and check box")
            self.status_bar("Please load csv files and check box", "red")
        elif any(checks == 0 for checks in self.checkbox_list):
            print("Please check boxes")
            self.status_bar("Please check boxes", "red")
        else:
            self.convert_datasets()
    
    # Write clean dataframes to JSON file and store in a local directory
    def convert_datasets(self):
        
        root_names = ["inventory", "inspections", "violations"]
        df_list = [self.df_inventory, self.df_inspections, self.df_violations]
        
        # Loop through all 3 dataset
        for i, dataset in enumerate(root_names):
            
            msg = "Writing {}.json...".format(dataset)
            print(msg)
            self.status_bar(msg, "white")
            jsonFilePath = "{}.json".format(dataset)
            
            # Write dataframe to JSON oreintated on records and indented for readability
            with open(jsonFilePath, 'w', encoding='utf-8') as f:
                f.write(df_list[i].to_json(orient='records', indent=2))

            msg = "Finshed converting {}.csv to {}.json".format(dataset, dataset)
            print(msg)
            self.status_bar(msg, "green")
            
        print("Finshed converting all datasets")
        self.status_bar("Finished converting all datasets", "green")
    
    # Connect to MongoDB using default local host and port settings
    def connect_to_db(self):
        
        # Connect to MongoDB
        print("Connecting to DB...")
        self.status_bar("Connecting to DB...", "white")
        self.client = pymongo.MongoClient("mongodb://localhost:27017/") 
        print("Connection Successful")
        
        # Create a new Database
        print("Building a database:'AP_Summative...'")
        self.db = self.client["AP_Summative"] 
        print("Database building successful")
        
        # Create 3 new collections within database
        print("Building collections: [inventory, inspections, violations]...")
        self.collection_inventory = self.db["inventory"]
        self.collection_inspections = self.db["inspections"]
        self.collection_violations = self.db["violations"]
        
        print("Collections building successfull")
        self.status_bar("Connection to DB successful", "green")
    
    # Send JSON data to MongoDB
    def send_to_db(self):
        
        # Check that connection exists
        if self.client == None:
            print("Please connect to Mongo DB first")
            self.status_bar("Convert data to Mongo DB first", "red")
        
        else:
            
            # Clear records from database
            self.status_bar("Sending JSON files to MongoDB", "white")
            print("Deleting old records")
            self.collection_inventory.delete_many({})
            self.collection_inspections.delete_many({})
            self.collection_violations.delete_many({})
            
            collections = [self.collection_inventory, self.collection_inspections, self.collection_violations]
            print(self.db.list_collection_names())
            datasets = ["inventory", "inspections", "violations"]
            
            # Loop through each collection inserting JSON records to MongoDB
            for i in range(3):

                jsonFilePath = "{}.json".format(datasets[i]) # Get relative filepath
                print("Opening JSON file {}.json...".format(datasets[i]))
                
                # Try and load file and throw error if it doesnt exist
                try:
                    inFile = open(jsonFilePath) # Open JSON file
                    dataFile = json.load(inFile) # Load JSON file using module

                    msg = "Writing {}.json to the Mongo DB...".format(datasets[i])
                    print(msg)
                    self.status_bar(msg, "white")

                    bulk_item_list = []
                    for item in dataFile:
                        bulk_item_list.append(item) # Read JSON records into an array
                    result = collections[i].insert_many(bulk_item_list) # Pymongo array to MongoDB collection

                    print("Finished loading {} dataset into the Mongo DB.".format(datasets[i]))
                    
                # Throw IO error if JSON cannot be loaded
                except IOError as err:
                    print("Error:", err)
                    print("Could not open '{}.json' file. Convert data to JSON first".format(datasets[i]))
                    self.status_bar("Could not open json file. Convert data to JSON first", "red")
            
            print("Finished uploading all datasets to the database")
            self.status_bar("Finished uploading all datasets to the database", "green")
    
    # Retreive Data from MongoDB and load into Pandas Dataframe
    def retrieve_from_db(self):
        
        # Check that connection to DB exists
        if self.client == None:
            print("Please connect to Mongo DB first")
            self.status_bar("Convert data to Mongo DB first", "red")
        
        else:
            # Query database Inventory collection to retrieve only the critical labels
            print("Retrieving Inventory.json from database...")
            self.status_bar("Retrieving Inventory.json from database...", "white")
            self.df_inventory = pd.DataFrame(list(self.collection_inventory.find({}, {
                '_id': 0,
                'FACILITY ID': 1,
                'FACILITY NAME': 1,
                'FACILITY ZIP': 1
            })))
            print("Inventory loaded.")
            
            # Query database Inspections collection to retrieve only the critical labels
            print("Retrieving Inspections.json from database...")
            self.status_bar("Retrieving Inspections.json from database...", "white")

            self.df_inspections = pd.DataFrame(list(self.collection_inspections.find({}, {
                '_id': 0,
                'FACILITY ID': 1,
                'FACILITY NAME': 1,
                'SERIAL NUMBER': 1,
                'PE DESCRIPTION': 1,
                'YEAR': 1,
                'SCORE': 1,
                'FACILITY ZIP': 1
                })))
            print("Inspections loaded.")
            
            # Query database Violations collection to retrieve only the critical labels
            print("Retrieving Violations.json from database...")
            self.status_bar("Retrieving Violations.json from database...", "white")
            self.df_violations = pd.DataFrame(list(self.collection_violations.find({}, {
                '_id': 0,
                'SERIAL NUMBER': 1,
                'VIOLATION CODE': 1,
                'POINTS': 1,
            })))
            print("Violations loaded.")
            
            print("Finished Retreiving Data from Database")
            self.status_bar("Finished Retreiving Data from Database", "green")
    
    # Display Dataframe header and first 100 rows in a table
    def preview_dataframes(self, n):
        self.n = n
        if self.n == 0:
            print("Previewing Inventory - first 100 lines")
            df = self.df_inventory.head(100) # Select first 100 rows of dataframe
            headers, data = self.convert_df_to_2Darray(df) # Converter function
            self.print_data(data, headers) # Print function to display data in GUI
        
        elif self.n == 1:
            print("Previewing Inspections - first 100 lines")
            df = self.df_inspections.head(100) # Select first 100 rows of dataframe
            headers, data = self.convert_df_to_2Darray(df) # Converter function
            self.print_data(data, headers) # Print function to display data in GUI
        elif self.n == 2:
            print("Previewing Violations - first 100 lines")
            df = self.df_violations.head(100) # Select first 100 rows of dataframe
            headers, data = self.convert_df_to_2Darray(df) # Converter function
            self.print_data(data, headers) # Print function to display data in GUI
    
    # Convert dataframe to a 2D numpy array
    def convert_df_to_2Darray(self, dataframe):
        self.dataframe = dataframe
        headers = list(self.dataframe.columns.values)
        numpy_array = self.dataframe.to_numpy()
        data = []
        for row in numpy_array:
            data.append(row.tolist())
        return headers, data
    
    # Calculate Seating Type stats. Mode, Median, Mean. Display in table.
    def seating_stats(self):
        
        # Group dataframe by PE_Descripton and Year and calc aggregate stats
        print("Calculating score stats for seating types...")
        self.df_groupby_seating = self.df_inspections.groupby(['PE DESCRIPTION', 'YEAR'])['SCORE'].agg([np.mean, np.median, lambda x: x.mode().iloc[0]])
        self.df_groupby_seating = self.df_groupby_seating.rename(columns={"<lambda_0>": "MODE", "mean": "MEAN", "median": "MEDIAN"}) # Rename headers
        self.df_groupby_seating['MEAN'] = self.df_groupby_seating['MEAN'].apply(lambda x: round(x, 2)) # Round to 2dp 
        self.df_groupby_seating['MEDIAN'] = self.df_groupby_seating['MEDIAN'].astype(int) # Convert to Ints
        self.df_groupby_seating = self.df_groupby_seating.reset_index() # Flatten the column names after grouping

        # convert dataframe in a list of column names and a python 2D array
        headers, data = self.convert_df_to_2Darray(self.df_groupby_seating)
                    
        self.print_data(data, headers)
        print("Score stats for seating types - Success")

    
    # Calculate Zipcode stats. Mode, Median, Mean. Display in table.
    def zipcode_stats(self):
        
        # Group dataframe by Zipcode and Year and calc aggregate stats
        print("Calculating score stats for zipcodes...")
        self.df_groupby_zip = self.df_inspections.groupby(['FACILITY ZIP', 'YEAR'])['SCORE'].agg([np.mean, np.median, lambda x: x.mode().iloc[0]])
        self.df_groupby_zip = self.df_groupby_zip.rename(columns={"<lambda_0>": "MODE", "mean": "MEAN", "median": "MEDIAN"}) # Rename headers
        self.df_groupby_zip['MEAN'] = self.df_groupby_zip['MEAN'].apply(lambda x: round(x, 2)) # Round to 2dp 
        self.df_groupby_zip['MEDIAN'] = self.df_groupby_zip['MEDIAN'].astype(int) # Convert to Ints
        self.df_groupby_zip = self.df_groupby_zip.reset_index() # Flatten the column names after grouping

        # convert dataframe in a list of column names and a python 2D array
        headers, data = self.convert_df_to_2Darray(self.df_groupby_zip)

        self.print_data(data, headers)
        print("Score stats for zipcodes - Success")

    # Print dataframes to GUI table, taking array of headers and a 2D array of data
    def print_data(self, data, headers):
        
        print("Displaying data in table view...")
        self.data = data
        self.columns = headers
        
        # Delete current table data
        self.my_tree.destroy()
        self.scrollbar.destroy()
        
        # Create Treeview Widget
        self.my_tree = ttk.Treeview(self.container_right, columns=self.columns, show="headings", height=30)

        # Format columns
        self.my_tree.column("#0", width=0, stretch=tk.NO) #hide phantom column
        for column in self.columns:
            self.my_tree.column(column, anchor=tk.W, width=100, stretch=0)
    
        # Create headings in the top bar
        self.my_tree.heading("#0", text="", anchor=tk.W) #hide phantom column
        for column in self.columns:
            self.my_tree.heading(column, text=column, anchor=tk.W)

        # Add data to table widget
        for count, record in enumerate(self.data):
            self.my_tree.insert(parent='', index='end', iid=count, text="", values=(record))

        # Pack to screen
        self.my_tree.pack(side="left", fill="y")
        
        # Add vertical scroll bar
        self.scrollbar = tk.Scrollbar(self.container_right, orient='vertical')
        self.scrollbar.configure(command=self.my_tree.yview)
        self.scrollbar.pack(side="right", fill="y")
        self.my_tree.config(yscrollcommand=self.scrollbar.set)
    
    # Generate plots of vendors and violation and open in pop-out window
    def plot_vendors_violations(self):
        print("Generating plot...")
        self.status_bar("Generating plot...", "white")
        
        # Merge datasets and group by violation code, counting unnique facilities
        df_merge = self.df_violations.merge(self.df_inspections, on='SERIAL NUMBER')
        df_nuniques = df_merge.groupby(['VIOLATION CODE'])['FACILITY ID'].nunique()

        # Print to table
        df_flat = df_nuniques.to_frame().reset_index()
        headers, data = self.convert_df_to_2Darray(df_flat)        
        self.print_data(data, headers)

        # Plot to popout window
        df_nuniques.plot(kind = 'bar',
              title = 'Distribution of Establishments commiting each type of Violation',
              figsize = (18, 6),
              logy=True)
        plt.xlabel("Violation Code")
        plt.ylabel("Number of Establishments")
        plt.show(block=False)

        print("Distribution of Establishments Plot - Success")
        self.status_bar("Distribution of Establishments Plot - Success", "green")
    
    # Generate plots of zipcodes and calculate one way ANOVA, open in pop-out window
    def plot_vendors_zipcodes(self):
        print("Generating plot...")
        self.status_bar("Generating plot...", "white")
        
        # Merge violations and inspections on key=SERIALNUMBER
        df_merge = self.df_violations.merge(self.df_inspections, on='SERIAL NUMBER')

        # Count the total violation points for each vendor
        df_violations_per_vendor = df_merge.groupby(['FACILITY ID'], as_index=False)['POINTS'].sum()

        # Clean up inventory zip codes
        self.df_inventory['temp'] = self.df_inventory['FACILITY ZIP'].str.extract(r"(\d{5})")
        del self.df_inventory['FACILITY ZIP']
        self.df_inventory = self.df_inventory.rename({'temp': 'FACILITY ZIP'}, axis=1)

        # Merge points with inventory on key=FACILITY ID
        df_zip_violations = self.df_inventory.merge(df_violations_per_vendor, on='FACILITY ID', how='left')

        # Rename column, remove NaNs and cast data types
        df_zip_violations = df_zip_violations.rename(columns={"POINTS": "TOTAL VIOLATIONS"})
        df_zip_violations['TOTAL VIOLATIONS'].fillna(0, inplace=True)
        df_zip_violations['TOTAL VIOLATIONS'] = df_zip_violations['TOTAL VIOLATIONS'].astype(int)

        # Calcualte ANOVA with manual function
        f = self.calculate_ANOVA(df_zip_violations, 'FACILITY ZIP', 'TOTAL VIOLATIONS')
        print("F-Value: {}: ".format(f))
        
        # Plot to popout window
        df_zip_violations.boxplot(by ='FACILITY ZIP', column =['TOTAL VIOLATIONS'], grid = False)
        plt.xlabel("Zipcodes")
        plt.ylabel("Violations per Vendor")
        msg = "ANOVA F-Value = {}.".format(round(f, 2))
        plt.suptitle("Box plot of Violations/Vendor grouped by Zipcodes")
        plt.title(msg)
        plt.show(block=False)

        print("Box plot of Violations/Vendor grouped by Zipcodes - Success")
        self.status_bar("Box plot of Violations/Vendors - Success - ANOVA F-Value = {}".format(round(f, 2)), "green")
    
    # Calcuate One Way ANOVA by passing in the Zipcode groups (catorgorical) and violation scores (continuous)
    def calculate_ANOVA(self, df, group, variable):

        self.df = df
        self.group = group
        self.variable = variable
    
        num_of_zipcodes = len(pd.unique(self.df[self.group]))  # number of zipcodes
        N = len(self.df.values)  # zipcodes x violations
        n = self.df.groupby(self.group).size().values #number of vendors per zipcode

        diff_between = num_of_zipcodes - 1
        diff_within = N - num_of_zipcodes

        # Sum of Squares Between
        sum_square_between = (sum(self.df.groupby(self.group).sum()[self.variable].values**2/n)) - (self.df[self.variable].sum()**2)/N

        # Sum of Squares Within
        sum_y_squared = sum([value**2 for value in self.df[self.variable].values])
        sum_square_within = sum_y_squared - sum(self.df.groupby(self.group).sum()[self.variable].values**2/n)

        # Calculation of Sum of Squares Total
        sum_square_total = sum_y_squared - (self.df[self.variable].sum()**2)/N

        # Calculate Mean Square Between
        mean_square_between = sum_square_between/diff_between

        # Calculation of Mean Square Within
        mean_square_within = sum_square_within/diff_within

        # Calculating the F-value
        f_value = mean_square_between/mean_square_within
        
        return f_value

root = tk.Tk() # Create tkinter GUI object
root.title("Summative App") #App title
app = App(root) #Main app class
root.mainloop() # Start Application

Connecting to DB...
Connection Successful
Building a database:'AP_Summative...'
Database building successful
Building collections: [inventory, inspections, violations]...
Collections building successfull
Retrieving Inventory.json from database...
Inventory loaded.
Retrieving Inspections.json from database...
Inspections loaded.
Retrieving Violations.json from database...
Violations loaded.
Finished Retreiving Data from Database
Previewing Inventory - first 100 lines
Displaying data in table view...
Calculating score stats for seating types...
Displaying data in table view...
Score stats for seating types - Success
Calculating score stats for zipcodes...
Displaying data in table view...
Score stats for zipcodes - Success
Previewing Inspections - first 100 lines
Displaying data in table view...
Previewing Violations - first 100 lines
Displaying data in table view...
Generating plot...
Displaying data in table view...
Distribution of Establishments Plot - Success
Generating plot...
F-Valu